In [ ]:
#creation of date table
CREATE TABLE dimDate (
    date_key INTEGER GENERATED ALWAYS AS IDENTITY PRIMARY KEY, 
    date DATE,
    year SMALLINT,
    quarter SMALLINT,
    month SMALLINT,
    day SMALLINT,
    week SMALLINT,
    is_weekend BOOLEAN
);

In [ ]:
SELECT 
    dimd.date_key,
    dimc.customer_key,
    dimm.movie_key,
    dims.store_key,
    p.amount
FROM payment p 
JOIN customer c 
ON c.customer_id = p.customer_id
JOIN dimcustomer dimc 
ON dimc.customer_id = c.customer_id
JOIN store s 
ON s.store_id = c.store_id
JOIN dimstore dims
ON s.store_id = dims.store_id
JOIN inventory i 
ON i.store_id = s.store_id 
JOIN film f     
ON f.film_id = i.film_id
JOIN dimmovie dimm
ON dimm.film_id = f.film_id
JOIN dimdate dimd
ON dimd.date_key = p.date_key

In [ ]:
DROP TABLE dimMovie

SELECT *
FROM dimdate

CREATE TABLE factSales
(
 sales_key SERIAL PRIMARY KEY,
 date_key INTEGER REFERENCES dimdate (date_key),
 customer_key INTEGER REFERENCES dimcustomer (customer_key),
 movie_key INTEGER REFERENCES dimmovie (movie_key),
 store_key INTEGER REFERENCES dimstore (store_key),
 sales_amount INTEGER
);

INSERT INTO factsales (
date_key,
customer_key,
movie_key,
store_key,
sales_amount
)
SELECT
dimdate.date_key,
dimcustomer.customer_key,
dimmovie.movie_key,
dimstore.store_key,
payment.amount
FROM sales
JOIN dimdate ON (sales.date_key = dimdate.date_key)
JOIN dimcustomer ON (sales.customer_key = dimcustomer.customer_key)
JOIN dimmovie ON (sales.movie_key = dimmovie.movie_key)
JOIN dimstore ON (sales.store_key = dimstore.store_key);

SELECT *
FROM payment



### To check your work, run the following query to see a table with the field names and data types. The output should match the table below.

In [ ]:
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name   = 'dimdate'

In [ ]:
CREATE TABLE dimCustomer
(
  customer_key SERIAL PRIMARY KEY,
  customer_id  smallint NOT NULL,
  first_name   varchar(45) NOT NULL,
  last_name    varchar(45) NOT NULL,
  email        varchar(50),
  address      varchar(50) NOT NULL,
  address2     varchar(50),
  district     varchar(20) NOT NULL,
  city         varchar(50) NOT NULL,
  country      varchar(50) NOT NULL,
  postal_code  varchar(10),
  phone        varchar(20) NOT NULL,
  active       smallint NOT NULL,
  create_date  timestamp NOT NULL,
  start_date   date NOT NULL,
  end_date     date NOT NULL
);

In [ ]:
CREATE TABLE dimMovie
(
  movie_key          SERIAL PRIMARY KEY,
  film_id            smallint NOT NULL,
  title              varchar(255) NOT NULL,
  description        text,
  release_year       year,
  language           varchar(20) NOT NULL,
  original_language  varchar(20),
  rental_duration    smallint NOT NULL,
  length             smallint NOT NULL,
  rating             varchar(5) NOT NULL,
  special_features   varchar(60) NOT NULL
);

In [ ]:
CREATE TABLE dimStore
(
  store_key           SERIAL PRIMARY KEY,
  store_id            smallint NOT NULL,
  address             varchar(50) NOT NULL,
  address2            varchar(50),
  district            varchar(20) NOT NULL,
  city                varchar(50) NOT NULL,
  country             varchar(50) NOT NULL,
  postal_code         varchar(10),
  manager_first_name  varchar(45) NOT NULL,
  manager_last_name   varchar(45) NOT NULL,
  start_date          date NOT NULL,
  end_date            date NOT NULL
);

In [ ]:
%%sql
CREATE TABLE factSales
(
 sales_key INTEGER REFERENCES ,
 date_key INTEGER REFERENCES dimDate (date_key),
 customer_key INTEGER REFERENCES dimCustomer (customer_key)
 movie_key INTEGER REFERENCES dimMovie (movie_key)
 store_key INTEGER REFERENCES dimStore (store_key)
 sales_amount INTEGER
);

In [ ]:
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name   = 'dimdate'

In [ ]:
INSERT INTO dimmovie(
    movie_key, 
    film_id, title, 
    description, 
    release_year, 
    language, 
    original_language, 
    rental_duration, 
    length, rating, 
    special_features
    )
SELECT
    film_id as movie_key,
    f.film_id,
    f.title,
    f.DESCRIPTION,
    f.release_year,
    l.name as language,
    f.original_language_id as original_language,
    f.rental_duration,
    f.length,
    f.rating,
    f.special_features
FROM film f    
JOIN language l
ON f.language_id = l.language_id

In [ ]:
INSERT INTO dimstore (
    store_key,
    store_id,
    address,
    address2,
    district,
    city,
    country,
    postal_code,
    manager_first_name,
    manager_last_name,
    start_date,
    end_date
)
SELECT DISTINCT
    s.store_id AS store_key,
    s.store_id, 
    a.address, 
    a.address2, 
    a.district, 
    c.city, 
    co.country, 
    a.postal_code, 
    st.first_name AS manager_first_name, 
    st.last_name AS manager_last_name, 
    now() AS start_date, 
    now() AS end_date
FROM store s
JOIN staff st ON st.store_id = s.store_id
JOIN address a ON a.address_id = s.address_id
JOIN city c ON c.city_id = a.city_id
JOIN country co ON co.country_id = c.country_id;

In [ ]:
INSERT INTO dimCustomer (
    customer_id, 
    first_name, 
    last_name, 
    email, 
    address, 
    address2, 
    district, 
    city, 
    country, 
    postal_code, 
    phone, 
    active, 
    create_date, 
    start_date, 
    end_date
)
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    a.address,
    a.address2,
    a.district,
    ci.city,
    co.country,
    a.postal_code,
    a.phone,
    c.active,
    c.create_date,
    now() AS start_date,
    now() AS end_date
FROM customer c
JOIN address a ON c.address_id = a.address_id
JOIN city ci ON a.city_id = ci.city_id
JOIN country co ON ci.country_id = co.country_id;

In [ ]:
SELECT 
    p.amount,
    c.customer_id,
    s.store_id,
    dims.store_key
FROM payment p 
JOIN customer c 
ON c.customer_id = p.customer_id
JOIN dimcustomer dimc 
ON dimc.customer_id = c.customer_id
JOIN store s 
ON s.store_id = c.store_id
JOIN dimstore dims
ON s.store_id = dims.store_id
JOIN inventory i 
ON i.store_id = s.store_id 
JOIN film f     
ON f.film_id = i.film_id
JOIN dimmovie dimm
ON dimm.film_id = f.film_id

In [ ]:
INSERT INTO dimDate (date_key, date, year, quarter, month, day, week, is_weekend)
SELECT DISTINCT(TO_CHAR(payment_date :: DATE, 'yyyyMMDD')::integer) AS date_key,
       date(payment_date)                                           AS date,
       EXTRACT(year FROM payment_date)                              AS year,
       EXTRACT(quarter FROM payment_date)                           AS quarter,
       EXTRACT(month FROM payment_date)                             AS month,
       EXTRACT(day FROM payment_date)                               AS day,
       EXTRACT(week FROM payment_date)                              AS week,
       CASE WHEN EXTRACT(ISODOW FROM payment_date) IN (6, 7) THEN true ELSE false END AS is_weekend
FROM payment;

In [ ]:
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name   = 'dimdate'